In [6]:
%load_ext autoreload 
%autoreload 2
import numpy as np
import random
from tqdm import tqdm
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')
from colossus.halo import concentration, mass_defs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from pylab import rcParams
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = r'\boldmath'
rcParams['font.family'] = 'serif'
rc('text.latex',preamble=r'\usepackage{/Users/kevinlevy/Documents/codes/master_thesis/code/apjfonts}')
from matplotlib.lines import Line2D
from cosmo import CosmoCalc
import sims
import foregrounds as fg
import experiments as exp
import tools
import lensing_estimator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Effect of Filters

In [2]:
nber_rand = 1
map_params = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
cluster = [9e15, 3, 1]
l, bl = exp.beam_power_spectrum(beam_fwhm = 1.4)
l, nl = exp.noise_power_spectrum(noiseval_white = 2.0)
cmb_map = sims.cmb_mock_data(map_params, l, cl, cluster, bl_arr = [bl], nl_arr = [nl])
l, nl_deconvolved =  exp.noise_power_spectrum(noiseval_white = 2.0, beam_fwhm = 1.4)
cl_noise = nl_deconvolved
wiener_filter = tools.wiener_filter(l, cl, cl_noise)
cmb_map_wiener_filtered = tools.convolve(cmb_map, l, wiener_filter, map_params = map_params) 
low_pass_filter = tools.low_pass_filter(l,  2000)
cmb_map_low_pass_filtered = tools.convolve(cmb_map_wiener_filtered, l, low_pass_filter, map_params = map_params) 

KeyboardInterrupt: 

In [ ]:
_, extent = tools.make_grid(map_params)
fig, ax = plt.subplots(1,3, figsize=(10,10))
sz = 20

im0 = ax[0].imshow(cmb_map[100:140, 100:140], extent = extent, cmap = cm.RdBu_r)
cb0 = fig.colorbar(im0, ax = ax[0], shrink = 0.228)
cb0.ax.tick_params(labelsize=sz-7)
ax[0].set_xlabel(r'$\theta_x\ $'+'[arcmin]', fontsize = sz)
ax[0].set_ylabel(r'$\theta_y\ $'+'[arcmin]', fontsize = sz)
ax[0].set_title('Unfiltered', fontsize = sz)
ax[0].tick_params(labelsize = sz-7)

im1 = ax[1].imshow(cmb_map_wiener_filtered[100:140, 100:140], extent = extent, cmap = cm.RdBu_r)
cb1 = fig.colorbar(im1, ax = ax[1], shrink = 0.228)
cb1.ax.tick_params(labelsize=sz-7)
ax[1].set_xlabel(r'$\theta_x\ $'+'[arcmin]', fontsize = sz)
ax[1].set_ylabel(r'$\theta_y\ $'+'[arcmin]', fontsize = sz)
ax[1].set_title('Wiener Filtered', fontsize = sz)
ax[1].get_yaxis().set_visible(False)
ax[1].tick_params(labelsize = sz-7)


im2 = ax[2].imshow(cmb_map_low_pass_filtered[100:140, 100:140], extent = extent, cmap = cm.RdBu_r)
cb2 = fig.colorbar(im1, ax = ax[2], shrink = 0.228)
cb2.set_label(label=r'$T \ [\mu$'+'K]', size=sz)
cb2.ax.tick_params(labelsize=sz-7)
ax[2].set_xlabel(r'$\theta_x\ $'+'[arcmin]', fontsize = sz)
ax[2].set_ylabel(r'$\theta_y\ $'+'[arcmin]', fontsize = sz)
ax[2].set_title('Low-Pass Filtered', fontsize = sz)
ax[2].get_yaxis().set_visible(False)
ax[2].tick_params(labelsize = sz-7)


fig.tight_layout(pad=0)
fig.savefig('effect_of_filters.eps', dpi = 200., bbox_inches = 'tight', pad_inches = 0.1)

## Pipeline

In [ ]:
nber_clus = 30000
nber_rand = 30000
map_params = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
c500 = concentration.concentration(2e14, '500c', 0.7)
M200c, _, c200c = mass_defs.changeMassDefinition(2e14, c500, 0.7, '500c', '200c', profile='nfw')
cluster = [M200c, c200c, 0.7]
fname = 'sim_data/mdpl2_cutouts_for_tszksz_clus_detection_M1.7e+14to2.3e+14_z0.6to0.8_15320haloes_boxsize20.0am.npz'
cutouts_dic = np.load(fname, allow_pickle = 1, encoding= 'latin1')['arr_0'].item()
mass_z_key = list(cutouts_dic.keys())[0]
cutouts = cutouts_dic[mass_z_key]
scale_fac = fg.compton_y_to_delta_Tcmb(freq = 145, uK = True)
tsz_ksz_cutouts = []
for kcntr, keyname in enumerate( cutouts ):
    tsz_cutout = cutouts[keyname]['y']*scale_fac
    ksz_cutout = cutouts[keyname]['ksz']*random.randrange(-1, 2, 2)
    tsz_ksz_cutout = tsz_cutout + ksz_cutout
    tsz_ksz_cutouts.append(tsz_ksz_cutout)  
l, bl = exp.beam_power_spectrum(beam_fwhm = 0)
l, nl = exp.noise_power_spectrum(noiseval_white = 0)
l, nl_deconvolved =  exp.noise_power_spectrum(noiseval_white = 0, beam_fwhm = 0)
cl_noise = nl_deconvolved


maps_clus = []
cutouts_clus = []
cutouts_clus_aligned = []
for i in tqdm(range(nber_clus)):
    maps_clus.append(sims.cmb_mock_data(map_params, l, cl, cluster, cluster_corr_cutouts_arr = [tsz_ksz_cutouts], bl_arr = [bl], nl_arr = [nl]))
    cutouts_clus.append(lensing_estimator.get_random_cutout(map_params, maps_clus[i]))
    cutouts_clus_aligned.append(lensing_estimator.get_aligned_cutout(map_params, maps_clus[i], l, cl, cl_noise)[0])
    

maps_rand = []
cutouts_rand = []
cutouts_rand_aligned = []
for i in tqdm(range(nber_rand)):
    maps_rand.append(sims.cmb_mock_data(map_params, l, cl, bl_arr = [bl], nl_arr = [nl]))
    cutouts_rand.append(lensing_estimator.get_random_cutout(map_params, maps_rand[i]))
    cutouts_rand_aligned.append(lensing_estimator.get_aligned_cutout(map_params, maps_rand[i], l, cl, cl_noise)[0])
    
    

bins, lensing_dipole_profile, stacks = lensing_estimator.get_dipole_profile(map_params, maps_clus, maps_rand,  l, cl, cl_noise, use_magnitude_weights = True, noise_weights = None, correct_for_tsz = True)


del maps_clus
del maps_rand

In [ ]:
_, extent = tools.make_grid([40, 0.25, 40, 0.25])
fig, axarr = plt.subplots(5,6, figsize=(10,10))


img00 = axarr[0,0].imshow(cutouts_clus[0], extent = extent, cmap = cm.RdBu_r)
axarr[0,0].get_xaxis().set_visible(False)
axarr[0,0].get_yaxis().set_visible(False)
img01 = axarr[0,1].imshow(cutouts_clus[1], extent = extent, cmap = cm.RdBu_r)
axarr[0,1].get_xaxis().set_visible(False)
axarr[0,1].get_yaxis().set_visible(False)
img02 = axarr[0,2].imshow(cutouts_clus[2], extent = extent, cmap = cm.RdBu_r)
axarr[0,2].get_xaxis().set_visible(False)
axarr[0,2].get_yaxis().set_visible(False)
img03 = axarr[0,3].imshow(cutouts_clus[3], extent = extent, cmap = cm.RdBu_r)
axarr[0,3].get_xaxis().set_visible(False)
axarr[0,3].get_yaxis().set_visible(False)
img04 = axarr[0,4].imshow(cutouts_clus[4], extent = extent, cmap = cm.RdBu_r)
axarr[0,4].get_xaxis().set_visible(False)
axarr[0,4].get_yaxis().set_visible(False)
img05 = axarr[0,5].imshow(cutouts_clus[5], extent = extent, cmap = cm.RdBu_r)
axarr[0,5].get_xaxis().set_visible(False)
axarr[0,5].get_yaxis().set_visible(False)



img10 = axarr[1,0].imshow(cutouts_clus_aligned[0], extent = extent, cmap = cm.RdBu_r)
axarr[1,0].get_xaxis().set_visible(False)
axarr[1,0].get_yaxis().set_visible(False)
img11 = axarr[1,1].imshow(cutouts_clus_aligned[1], extent = extent, cmap = cm.RdBu_r)
axarr[1,1].get_xaxis().set_visible(False)
axarr[1,1].get_yaxis().set_visible(False)
img12 = axarr[1,2].imshow(cutouts_clus_aligned[2], extent = extent, cmap = cm.RdBu_r)
axarr[1,2].get_xaxis().set_visible(False)
axarr[1,2].get_yaxis().set_visible(False)
img13 = axarr[1,3].imshow(cutouts_clus_aligned[3], extent = extent, cmap = cm.RdBu_r)
axarr[1,3].get_xaxis().set_visible(False)
axarr[1,3].get_yaxis().set_visible(False)
img14 = axarr[1,4].imshow(cutouts_clus_aligned[4], extent = extent, cmap = cm.RdBu_r)
axarr[1,4].get_xaxis().set_visible(False)
axarr[1,4].get_yaxis().set_visible(False)
img15 = axarr[1,5].imshow(cutouts_clus_aligned[5], extent = extent, cmap = cm.RdBu_r)
axarr[1,5].get_xaxis().set_visible(False)
axarr[1,5].get_yaxis().set_visible(False)


img20 = axarr[2,0].imshow(cutouts_rand[0], extent = extent, cmap = cm.RdBu_r)
axarr[2,0].get_xaxis().set_visible(False)
axarr[2,0].get_yaxis().set_visible(False)
img21 = axarr[2,1].imshow(cutouts_rand[1], extent = extent, cmap = cm.RdBu_r)
axarr[2,1].get_xaxis().set_visible(False)
axarr[2,1].get_yaxis().set_visible(False)
img22 = axarr[2,2].imshow(cutouts_rand[2], extent = extent, cmap = cm.RdBu_r)
axarr[2,2].get_xaxis().set_visible(False)
axarr[2,2].get_yaxis().set_visible(False)
img23 = axarr[2,3].imshow(cutouts_rand[3], extent = extent, cmap = cm.RdBu_r)
axarr[2,3].get_xaxis().set_visible(False)
axarr[2,3].get_yaxis().set_visible(False)
img24 = axarr[2,4].imshow(cutouts_rand[4], extent = extent, cmap = cm.RdBu_r)
axarr[2,4].get_xaxis().set_visible(False)
axarr[2,4].get_yaxis().set_visible(False)
img25 = axarr[2,5].imshow(cutouts_rand[5], extent = extent, cmap = cm.RdBu_r)
axarr[2,5].get_xaxis().set_visible(False)
axarr[2,5].get_yaxis().set_visible(False)


img30 = axarr[3,0].imshow(cutouts_rand_aligned[0], extent = extent, cmap = cm.RdBu_r)
axarr[3,0].get_xaxis().set_visible(False)
axarr[3,0].get_yaxis().set_visible(False)
img31 = axarr[3,1].imshow(cutouts_rand_aligned[1], extent = extent, cmap = cm.RdBu_r)
axarr[3,1].get_xaxis().set_visible(False)
axarr[3,1].get_yaxis().set_visible(False)
img32 = axarr[3,2].imshow(cutouts_rand_aligned[2], extent = extent, cmap = cm.RdBu_r)
axarr[3,2].get_xaxis().set_visible(False)
axarr[3,2].get_yaxis().set_visible(False)
img33 = axarr[3,3].imshow(cutouts_rand_aligned[3], extent = extent, cmap = cm.RdBu_r)
axarr[3,3].get_xaxis().set_visible(False)
axarr[3,3].get_yaxis().set_visible(False)
img34 = axarr[3,4].imshow(cutouts_rand_aligned[4], extent = extent, cmap = cm.RdBu_r)
axarr[3,4].get_xaxis().set_visible(False)
axarr[3,4].get_yaxis().set_visible(False)
img35 = axarr[3,5].imshow(cutouts_rand_aligned[5], extent = extent, cmap = cm.RdBu_r)
axarr[3,5].get_xaxis().set_visible(False)
axarr[3,5].get_yaxis().set_visible(False)


img40 = axarr[4,0].imshow(stacks[0], extent = extent, cmap = cm.RdBu_r)
axarr[4,0].get_xaxis().set_visible(False)
axarr[4,0].get_yaxis().set_visible(False)
img41 = axarr[4,1].imshow(stacks[1], extent = extent, cmap = cm.RdBu_r)
axarr[4,1].get_xaxis().set_visible(False)
axarr[4,1].get_yaxis().set_visible(False)
img42 = axarr[4,2].imshow(stacks[2], extent = extent, cmap = cm.RdBu_r)
axarr[4,2].get_xaxis().set_visible(False)
axarr[4,2].get_yaxis().set_visible(False)
img43 = axarr[4,3].imshow(stacks[3], extent = extent, cmap = cm.RdBu_r)
axarr[4,3].get_xaxis().set_visible(False)
axarr[4,3].get_yaxis().set_visible(False)
img44 = axarr[4,4].imshow(stacks[4], extent = extent, cmap = cm.RdBu_r)
axarr[4,4].get_xaxis().set_visible(False)
axarr[4,4].get_yaxis().set_visible(False)
axarr[4,5].set_visible(False)


fig, ax = plt.subplots(1,1, figsize=(5.5,3.5))
ax.plot(bins, lensing_dipole_profile, color = 'black', ls = '', marker = '.')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

plt.tight_layout(pad=0)
plt.savefig('pipeline.eps', dpi = 200., bbox_inches = 'tight', pad_inches = 0.1)

## Number of Maps for Fitting

In [ ]:
nber_rand = 30000
map_params = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
cluster = [6e14, 3, 1]
l, nl_deconvolved =  exp.noise_power_spectrum(noiseval_white = 0, beam_fwhm = 0)
cl_noise = nl_deconvolved


maps_rand = []
for i in tqdm(range(nber_rand)):
    maps_rand.append(sims.cmb_mock_data(map_params, l, cl))


cutouts_aligned_bg, magnitude_weights = [], []
for i in tqdm(range(nber_rand)):
    cutout_aligned_bg, magnitude_weight = lensing_estimator.get_aligned_cutout(map_params, maps_rand[i], l, cl, cl_noise)
    cutouts_aligned_bg.append(cutout_aligned_bg)
    magnitude_weights.append(magnitude_weight)
stack_bg = lensing_estimator.get_stack(cutouts_aligned_bg, magnitude_weights)


nber_clus = 1000
dipole_profiles_1000 = []
for i in tqdm(range(25)):
    maps_clus = []
    for j in tqdm(range(nber_clus)):
        maps_clus.append(sims.cmb_mock_data(map_params, l, cl, cluster))
    cutouts_aligned, magnitude_weights = [], []
    for k in tqdm(range(nber_clus)):
        cutout_aligned, magnitude_weight = lensing_estimator.get_aligned_cutout(map_params, maps_clus[k], l, cl, cl_noise)
        cutouts_aligned.append(cutout_aligned)
        magnitude_weights.append(magnitude_weight)
    stack_clus = lensing_estimator.get_stack(cutouts_aligned, magnitude_weights)
    stack_dipole = stack_clus-stack_bg
    dipole_profile = np.mean(stack_dipole, axis = 0)   
    dipole_profiles_1000.append(dipole_profile)
del maps_clus
    
    
nber_clus = 10000
dipole_profiles_10000 = []
for i in tqdm(range(25)):
    maps_clus = []
    for j in tqdm(range(nber_clus)):
        maps_clus.append(sims.cmb_mock_data(map_params, l, cl, cluster))
    cutouts_aligned, magnitude_weights = [], []
    for k in tqdm(range(nber_clus)):
        cutout_aligned, magnitude_weight = lensing_estimator.get_aligned_cutout(map_params, maps_clus[k], l, cl, cl_noise)
        cutouts_aligned.append(cutout_aligned)
        magnitude_weights.append(magnitude_weight)
    stack_clus = lensing_estimator.get_stack(cutouts_aligned, magnitude_weights)
    stack_dipole = stack_clus-stack_bg
    dipole_profile = np.mean(stack_dipole, axis = 0)   
    dipole_profiles_10000.append(dipole_profile)
del maps_clus

    
nber_clus = 20000
dipole_profiles_20000 = []
for i in tqdm(range(25)):
    maps_clus = []
    for j in tqdm(range(nber_clus)):
        maps_clus.append(sims.cmb_mock_data(map_params, l, cl, cluster))
    cutouts_aligned, magnitude_weights = [], []
    for k in tqdm(range(nber_clus)):
        cutout_aligned, magnitude_weight = lensing_estimator.get_aligned_cutout(map_params, maps_clus[k], l, cl, cl_noise)
        cutouts_aligned.append(cutout_aligned)
        magnitude_weights.append(magnitude_weight)
    stack_clus = lensing_estimator.get_stack(cutouts_aligned, magnitude_weights)
    stack_dipole = stack_clus-stack_bg
    dipole_profile = np.mean(stack_dipole, axis = 0)   
    dipole_profiles_20000.append(dipole_profile)
del maps_clus
    

nber_clus = 30000
dipole_profiles_30000 = []
for i in tqdm(range(25)):
    maps_clus = []
    for j in tqdm(range(nber_clus)):
        maps_clus.append(sims.cmb_mock_data(map_params, l, cl, cluster))
    cutouts_aligned, magnitude_weights = [], []
    for k in tqdm(range(nber_clus)):
        cutout_aligned, magnitude_weight = lensing_estimator.get_aligned_cutout(map_params, maps_clus[k], l, cl, cl_noise)
        cutouts_aligned.append(cutout_aligned)
        magnitude_weights.append(magnitude_weight)
    stack_clus = lensing_estimator.get_stack(cutouts_aligned, magnitude_weights)
    stack_dipole = stack_clus-stack_bg
    dipole_profile = np.mean(stack_dipole, axis = 0)   
    dipole_profiles_30000.append(dipole_profile)
del maps_clus


del maps_rand

In [ ]:
fig, axs = plt.subplots(2, 2, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}, figsize=(9,6))
sz = 28

(ax1, ax2), (ax3, ax4) = axs
for i in range(24):
    ax1.plot(bins, dipole_profiles_1000[i], color = 'red', alpha = 0.1)
ax1.plot(bins, dipole_profiles_1000[24], color = 'red', alpha = 0.1, label = nber_clusters_int[0])
ax1.plot(bins, np.mean(dipole_profiles_1000, axis = 0), color = 'green', ls = '--', label = nber_clusters_int[0])
ax1.set_ylabel(r'$\langle\Delta T_{\theta_x}\rangle$'+r'$\ [\mu$'+'K]', fontsize = sz)
ax1.tick_params(labelsize = sz-7)
ax1.legend(ncol = 1, prop={'size': sz-12})
for i in range(24):
    ax2.plot(bins, dipole_profiles_10000[i], color = 'red', alpha = 0.1)
ax2.plot(bins, dipole_profiles_10000[24], color = 'red', alpha = 0.1, label = nber_clusters_int[1])
ax2.plot(bins, np.mean(dipole_profiles_10000, axis = 0), color = 'green', ls = '--')
ax2.tick_params(labelsize = sz-7)
ax2.legend(ncol = 1, prop={'size': sz-12})
for i in range(24):
    ax3.plot(bins, dipole_profiles_20000[i], color = 'red', alpha = 0.1)
ax3.plot(bins, dipole_profiles_20000[24], color = 'red', alpha = 0.1, label = nber_clusters_int[2])
ax3.plot(bins, np.mean(dipole_profiles_20000, axis = 0), color = 'green', ls = '--')
ax3.set_xlabel(r'$\theta_x$ [arcmin]', fontsize = sz)
ax3.set_ylabel(r'$\langle\Delta T_{\theta_x}\rangle$'+r'$\ [\mu$'+'K]', fontsize = sz)
ax3.set_xticks([-4,-2,0,2, 4])
ax3.tick_params(labelsize = sz-7)
ax3.legend(ncol = 1, prop={'size': sz-12})
for i in range(24):
    ax4.plot(bins, dipole_profiles_30000[i], color = 'red', alpha = 0.1)
ax4.plot(bins, dipole_profiles_30000[24], color = 'red', alpha = 0.1, label = nber_clusters_int[3])
ax4.plot(bins, np.mean(dipole_profiles_40000, axis = 0), color = 'green', ls = '--')
ax4.set_xlabel(r'$\theta_x$ [arcmin]', fontsize = sz)
ax4.set_xticks([-4,-2,0,2, 4])
ax4.tick_params(labelsize = sz-7)
ax4.legend(ncol = 1, prop={'size': sz-12})

fig.savefig('number_of_maps_for_fitting.eps', dpi = 200.)